<a href="https://colab.research.google.com/github/MahdiFaourr/MahdiFaourr/blob/main/HatOn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Collect and Prepare Data


In [ ]:
json_filepath_test = '/content/drive/MyDrive/Dataset/test/_annotations.coco.json'
imgs_filepath_test = '/content/drive/MyDrive/Dataset/test'

json_filepath_train = '/content/drive/MyDrive/Dataset/train/_annotations.coco.json'
imgs_filepath_train = '/content/drive/MyDrive/Dataset/train/'

In [ ]:
# read the file
with open(json_filepath_train, 'r') as file:
  annotations_data = json.load(file)

with open(json_filepath_test, 'r') as file:
  annotations_data_test = json.load(file)

#print(json.dumps(annotations_data, indent=2))
annotations = annotations_data.get('annotations', [])
annotations_test = annotations_data_test.get('annotations', [])
#print(annotations)

# Extract categories
categories = annotations_data['categories']
print(categories)

#2 is true else is false
dict_labels_binary = {category['id']: 0 if category['name'] != 'helmet' else 1 for category in categories}
print(dict_labels_binary)



In [ ]:
# Add binary labels to each annotation
#for annotation in annotations:
 #   category_id = annotation.get('category_id', -1)
  #  annotation['category_id'] = dict_labels_binary.get(category_id, -1)

#inspect
for annotation in annotations:
  print(annotation)

#note:[x, y, width, heightt ex: bbox starts at (214, 198) and has a width of 19 pixels and a height of 22 pixels.

In [ ]:
import os

# Specify the path for the test dataset within Colab's writable space
train_dataset_dir_colab = '/content/drive/MyDrive/Data/Train'
test_dataset_dir_colab = '/content/drive/MyDrive/Data/Test'


# Specify the class names
class_names = ['without_hard_hat', 'with_hard_hat']

# Construct paths for 'without_hard_hat' and 'with_hard_hat' directories within Colab
without_hat_dir_colab_train = os.path.join(train_dataset_dir_colab, class_names[0])
with_hat_dir_colab_train = os.path.join(train_dataset_dir_colab, class_names[1])

# Construct paths for 'without_hard_hat' and 'with_hard_hat' directories within Colab
without_hat_dir_colab_test = os.path.join(test_dataset_dir_colab, class_names[0])
with_hat_dir_colab_test = os.path.join(test_dataset_dir_colab, class_names[1])

# Create the directories if they don't exist
os.makedirs(without_hat_dir_colab_train, exist_ok=True)
os.makedirs(with_hat_dir_colab_train, exist_ok=True)
os.makedirs(without_hat_dir_colab_test, exist_ok=True)
os.makedirs(with_hat_dir_colab_test, exist_ok=True)


# Separate Images in Sub folders

In [ ]:
import os
import shutil

# Specify the source directory containing the photos
source_photos_dir = imgs_filepath_train
#then run on imgs_filepath_train again and change bellow accordignly

# Check if the source directory exists
if not os.path.exists(source_photos_dir):
    print(f"Source directory '{source_photos_dir}' does not exist.")
    exit()

# Assuming you have the annotations data available
images = annotations_data.get('images', [])
annotations = annotations_data.get('annotations', [])  # Assuming annotations is a list of dictionaries


# Populate the dictionary with data
for image in images:
    file_name = image['file_name']
    source_path = os.path.join(source_photos_dir, file_name)
    ID = image['id']

    # Find annotations for the current image ID
    similar_annotations_list = [ann for ann in annotations if ann.get('image_id') == ID]

    # Extract category_ids for the current image
    category_ids_list = [ann.get('category_id', -1) for ann in similar_annotations_list]

    # Check if category_id is 2 or if 2 is in the category_ids list
    if 2 in category_ids_list:
        destination_path = os.path.join(with_hat_dir_colab_train, file_name)
        shutil.copy(source_path, destination_path)
    else:
        destination_path = os.path.join(without_hat_dir_colab_train, file_name)
        shutil.copy(source_path, destination_path)



# Exploratory Data Analysis

In [ ]:
import pandas as pd
# Read JSON data for Test Data
df_1 = pd.json_normalize(annotations, sep='_').fillna(pd.NA)
print(df_1.head(10))
print()
print('The test file annotations has {} rows and {} columns'.format(df_1.shape[0],df_1.shape[1]))

#check for null values
print()
print(df_1.isnull().sum())


In [ ]:
# check JSON data for Train Data
df_2 = pd.json_normalize(annotations_test, sep='_').fillna(pd.NA)
print(df_2.head(10))
print()
print('The test file annotations has {} rows and {} columns'.format(df_2.shape[0],df_2.shape[1]))

#check for null values
print()
print(df_2.isnull().sum())




1.  **We have 7 attributes in total.**
2.   **Images have 1 or more than 1 annotation for every image.**
3. **There are no Null values in the data**





In [ ]:
#Number of Images in test and train files
Number_of_Images_Train = df_1['image_id'].unique()
print('There are {} number of Images in the Train file.'.format(len(Number_of_Images_Train)))

Number_of_Images_Test = df_2['image_id'].unique()
print('There are {} number of Images in the Test file.'.format(len(Number_of_Images_Test)))


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

category_mapping = {
    0:'Workers',
    1: 'head',
    2: 'helmet',
    3: 'person'
}
# Create a bar chart of 'category_id' counts
category_counts = df_1['category_id'].value_counts()
category_counts_test = df_2['category_id'].value_counts()

# Map category IDs to names
category_names = category_counts.index.map(category_mapping)
category_names_test = category_counts_test.index.map(category_mapping)

plt.figure(figsize=(3,2))
category_counts.index = category_names
category_counts.plot(kind='bar')
plt.xlabel('Categories')
plt.ylabel('Occurrences in Train')
plt.title('Occurrences of each category')
plt.show()


plt.figure(figsize=(3,2))
category_counts_test.index = category_names_test
category_counts_test.plot(kind='bar')
plt.xlabel('Categories')
plt.ylabel('Occurrences in Test')
plt.title('Occurrences of each category')
plt.show()


**We see almost the same distribution of cases among each category which shows balance between Test and Train files**

In [ ]:
import os
import matplotlib.pyplot as plt

test_dir = "/content/drive/MyDrive/Data/Test"
train_dir = "/content/drive/MyDrive/Data/Train"

# Initialize counters for test set
with_hat_count_test = len(os.listdir(os.path.join(test_dir, "with_hard_hat")))
without_hat_count_test = len(os.listdir(os.path.join(test_dir, "without_hard_hat")))

# Initialize counters for training set
with_hat_count_train = len(os.listdir(os.path.join(train_dir, "with_hard_hat")))
without_hat_count_train = len(os.listdir(os.path.join(train_dir, "without_hard_hat")))

# Bar chart for test set
labels_test = ['With Hat', 'Without Hat']
counts_test = [with_hat_count_test, without_hat_count_test]

# Bar chart for training set
labels_train = ['With Hat', 'Without Hat']
counts_train = [with_hat_count_train, without_hat_count_train]

# Create subplots with two columns
fig, axs = plt.subplots(1, 2, figsize=(10, 4))

# Plot test set bar chart with numbers
bars_test = axs[0].bar(labels_test, counts_test, color=['blue', 'red'])
axs[0].set_title('Test File')
axs[0].set_xlabel('Categories')
axs[0].set_ylabel('Number of Images')

for bar, count in zip(bars_test, counts_test):
    axs[0].text(bar.get_x() + bar.get_width() / 2 - 0.15, bar.get_height() + 0.1, str(count), ha='center', va='bottom')

# Plot training set bar chart with numbers
bars_train = axs[1].bar(labels_train, counts_train, color=['blue', 'red'])
axs[1].set_title('Train File')
axs[1].set_xlabel('Categories')
axs[1].set_ylabel('Number of Images')

for bar, count in zip(bars_train, counts_train):
    axs[1].text(bar.get_x() + bar.get_width() / 2 - 0.15, bar.get_height() + 0.1, str(count), ha='center', va='bottom')

plt.show()


# preprocessing Data



*   Utilize ImageDataGenerator to preprocess and augment the training images.
*   Create separate generators for training and validation data.

*  Flow the images in batches using generators with specified augmentation parameters.











In [ ]:
# using ImageDataGenerator for preprocessig and augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the paths for your training and validation datasets
train_data_dir = '/content/drive/MyDrive/Data/Train'
test_data_dir = '/content/drive/MyDrive/Data/Test'

# Define the image size and batch size
img_width, img_height =224,224
batch_size = 32

# Create an ImageDataGenerator for training data
datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Flow training images in batches using the train_datagen generator
train_iterator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

# Flow validation images in batches using the validation_datagen generator
test_iterator = datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

# Model Architecture and Training
*  Mini VGG is widely used in detection projects
*  MobileNet has highly performance and is more flexible and light to adapt with different devices
* Xception is known for its depth-wise separable convolutions and adapts to different input shapes

# 1. Mini VGG

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

def create_mini_vgg():
    model = Sequential()

    # Convolutional Block 1
    model.add(Conv2D(12, (3, 3), activation='relu', input_shape=(224,224,3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Convolutional Block 2
    model.add(Conv2D(24, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Convolutional Block 3
    model.add(Conv2D(12, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Fully Connected layers
    model.add(Flatten())
    model.add(Dense(125, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    return model
model=create_mini_vgg()
model.summary()

In [ ]:
from keras.metrics import Precision
model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy', Precision()])

# Train the model
model.fit(train_iterator,epochs=10,batch_size=32,verbose=1, validation_data=test_iterator)


Epoch 1/10
165/165 [==============================] - 351s 2s/step - loss: 0.2740 - accuracy: 0.9121 - precision: 0.0938 - val_loss: 0.2477 - val_accuracy: 0.9083 - val_precision: 0.0000e+00
Epoch 2/10
165/165 [==============================] - 372s 2s/step - loss: 0.2281 - accuracy: 0.9171 - precision: 0.0000e+00 - val_loss: 0.2406 - val_accuracy: 0.9083 - val_precision: 0.0000e+00
Epoch 3/10
165/165 [==============================] - 323s 2s/step - loss: 0.2175 - accuracy: 0.9171 - precision: 0.0000e+00 - val_loss: 0.2508 - val_accuracy: 0.9083 - val_precision: 0.0000e+00
Epoch 4/10
165/165 [==============================] - 320s 2s/step - loss: 0.2080 - accuracy: 0.9171 - precision: 0.0000e+00 - val_loss: 0.2447 - val_accuracy: 0.9083 - val_precision: 0.0000e+00
Epoch 5/10
165/165 [==============================] - 352s 2s/step - loss: 0.2150 - accuracy: 0.9171 - precision: 0.0000e+00 - val_loss: 0.2299 - val_accuracy: 0.9083 - val_precision: 0.0000e+00
Epoch 6/10
165/165 [=========

In [ ]:
model.evaluate(test_iterator)

56/56 [==============================] - 57s 1s/step - loss: 0.2394 - accuracy: 0.9083 - precision: 0.0000e+00


[0.2393515557050705, 0.9082672595977783, 0.0]

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import save_model
tf.keras.models.save_model(model, '/content/Trained_modelOne.keras')

In [ ]:
import pickle
with open('modelOne.sav', 'wb') as file:
    pickle.dump(model, file)

# 2. MobileNet_V1

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input, decode_predictions
import numpy as np

# Load MobileNetV1 base model without top (fully connected) layers
#base_model = MobileNet(weights='imagenet', include_top=False)
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


# Create a Sequential model
model = Sequential()

# Add the MobileNetV1 base model to the Sequential model
model.add(base_model)
# Set MobileNetV1 layers as non-trainable
for layer in base_model.layers:
    layer.trainable = False

# Add GlobalAveragePooling2D layer to reduce spatial dimensions
model.add(GlobalAveragePooling2D())

# Add custom dense layers for classification
model.add(Dense(220, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Print model summary to see the architecture
model.summary()
from keras.callbacks import Callback
from keras.metrics import Precision
# Display the model summary
model.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy', Precision()])

# Train the model with the custom callback
model.fit(train_iterator,
                   epochs=10,  # You may set this to a large number or use a loop with early stopping
                   batch_size=32,
                   verbose=1,
                   validation_data=test_iterator)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_224 (Functi  (None, 7, 7, 1024)        3228864   
 onal)                                                           
                                                                 
 global_average_pooling2d_1  (None, 1024)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_4 (Dense)             (None, 220)               225500    
                                                                 
 dense_5 (Dense)             (None, 1)                 221       
                                                                 
Total params: 3454585 (13.18 MB)
Trainable params: 225721 (881.72 KB)
Non-trainable params: 3228864 (12.32 MB)
_________________________________________________________________
Epoch 1/1

In [ ]:
model.evaluate(test_iterator)

56/56 [==============================] - 108s 2s/step - loss: 0.1115 - accuracy: 0.9473 - precision_2: 0.7091


[0.11151658743619919, 0.9473386406898499, 0.7090908885002136]

In [ ]:
tf.keras.models.save_model(model, filepath='/content/modelTwo.keras')

In [ ]:
with open('modelTwo.sav', 'wb') as file:
    pickle.dump(model, file)

# 3. Xception Pretrained Model

In [ ]:
#model test
import tensorflow as tf
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.metrics import Precision

# Load the pre-trained Xception model without the top layers
base_model = Xception(weights='imagenet', include_top=False, input_shape=(416,416, 3))

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Add your custom top layers for your specific task
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output_layer = Dense(1, activation='sigmoid')(x)  # Adjust NUM_CLASSES based on your task
# Create the new model
model = Model(inputs=base_model.input, outputs=output_layer)
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',Precision()])

# Train the model
model.fit(train_iterator, steps_per_epoch=len(train_iterator), epochs=10, validation_data=test_iterator, validation_steps=len(test_iterator))


In [ ]:
model.evaluate(test_iterator)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import save_model
tf.keras.models.save_model(model, filepath='/content/modelThree.keras')

# Testing Model in Colab

In [ ]:
from google.colab import files
from PIL import Image

# Upload image
uploaded_file = files.upload()

# Read the uploaded image
image_path = list(uploaded_file.keys())[0]
image = Image.open(image_path)


Saving 005302_jpg.rf.3f9ff0dcbfa01ac541e96d7aa79685ff.jpg to 005302_jpg.rf.3f9ff0dcbfa01ac541e96d7aa79685ff.jpg


In [ ]:
# Preprocess the image
image = image.resize((416, 416))  # Adjust size based on your model's input size
image_array = np.array(image)
preprocessed_image = image_array / 255.0  # Normalize pixel values to [0, 1]

# Expand dimensions to create a batch size of 1
preprocessed_image = np.expand_dims(preprocessed_image, axis=0)


In [ ]:
from scipy.special import softmax

# Make predictions

loaded_model = pickle.load(open('/content/modelOne.sav', 'rb'))
prediction = loaded_model.predict(preprocessed_image)
print(prediction)

if prediction[0][0] >= 0.5:
  print('Hard-Hat detected', prediction[0][0])
else:
  print('No Hard-Hat detected!', prediction[0][0])

